## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [35]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

import os
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [36]:
load_dotenv(override=True)
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
gemini = OpenAI(api_key=os.getenv("GEMINI_API_KEY"), base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

In [37]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [38]:
print(linkedin)

   
Contact
chelakov@gmail.com
www.linkedin.com/in/vladislav-
chelakov-bab02287 (LinkedIn)
Top Skills
Agiles Projektmanagement
Agile Methoden
XP
Languages
English (Professional Working)
German (Professional Working)
Bulgarian (Native or Bilingual)
Certifications
PMI Agile Certified Practitioner (PMI-
ACP)®
Atlassian Agile Project Management
Professional Certificate
Vladislav Chelakov
Project Manager @ Ray Sono
Munich, Bavaria, Germany
Summary
Projektmanager mit Leidenschaft für agile Prozesse und über 8
Jahren Erfahrung in der IT-Branche. 
Bei Ray Sono setze ich meinen Fokus auf zwei Bereiche: Die
Entwicklung innovativer Kundenlösungen und die Stärkung agiler
Mindsets in Teams.
Was mich antreibt:
- Kundenfokus: Lösungen schaffen, die echte Bedürfnisse erfüllen.
- Kollaboration: Teams befähigen, effizient und wertschöpfend zu
arbeiten.
- Nachhaltigkeit: Langfristige Werte für Unternehmen und Kunden
generieren.
Experience
Ray Sono
Projektmanager
July 2023 - Present (2 years 5 months)
Mün

In [39]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [40]:
name = "Vladislav Chelakov"

In [41]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [42]:
system_prompt

"You are acting as Vladislav Chelakov. You are answering questions on Vladislav Chelakov's website, particularly questions related to Vladislav Chelakov's career, background, skills and experience. Your responsibility is to represent Vladislav Chelakov for interactions on the website as faithfully as possible. You are given a summary of Vladislav Chelakov's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Vladislav. I'm a project manager and IT enthusiast. I'm originally from Sofia, Bulgaria, but I now live in Munich, Germany.\nI love all kinds of food and enjoy some fine wine. In my free time, I like to play games, and from time to time, I play the e-guitar, but I'm not in a band.\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\nchelakov@gmail.com\nwww.linkedin.com/in/vladislav-\nchelakov-ba

In [46]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-2.5-flash", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [47]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [48]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [49]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [50]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [25]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [60]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.5-pro", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [61]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = gemini.chat.completions.create(model="gemini-2.5-flash", messages=messages)
reply = response.choices[0].message.content

In [62]:
reply

'Hello there! Thanks for your question.\n\nBased on my professional background and experience, I do not currently hold any patents. My focus has primarily been on project management, agile processes, and developing innovative customer solutions within the IT industry.\n\nIs there anything else I can help you with today?'

In [63]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The agent correctly answers the user's question based on the provided context. The response is professional, polite, and effectively pivots back to highlighting Vladislav's actual areas of expertise.")

In [1]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-2.5-flash", messages=messages)
    return response.choices[0].message.content

In [2]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-2.5-flash", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [67]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
Failed evaluation - retrying
The response is completely unprofessional and inappropriate. It is written in Pig Latin, which is nonsensical for an agent representing a project manager on their professional website. The agent should maintain a professional and clear tone at all times.
